In [5]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AutoTokenizer, AutoModel 
import pickle

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")
# Example input texts (batch)
texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "The service was fantastic!",
    "Not worth the price.",
    "I would recommend this!"
]

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Forward pass through the first two layers
model_part1 = tinybert.encoder.layer[:2]

with torch.no_grad():
    # Get the initial hidden states from the embedding layer
    hidden_states = tinybert.embeddings(inputs['input_ids'])
    
    # Pass through the first two layers
    for layer in model_part1:
        hidden_states = layer(hidden_states)[0]  # Get the output from each layer

# Serialize and send the intermediate tensor to the server
data = pickle.dumps(hidden_states)
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('172.16.19.59', 10300))  # Adjust host and port as needed
client_socket.sendall(data)
client_socket.close()
print("Intermediate output sent to server.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


AttributeError: 'BertForSequenceClassification' object has no attribute 'encoder'

In [1]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./optimized model")
tokenizer = BertTokenizer.from_pretrained("./optimized model")

# Example input texts (batch)
texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "The service was fantastic!",
    "Not worth the price.",
    "I would recommend this!"
]

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Forward pass through the first two layers
model_part1 = tinybert.bert.encoder.layer[:2]  # Access the encoder correctly

with torch.no_grad():
    # Get the initial hidden states from the embedding layer
    hidden_states = tinybert.bert.embeddings(inputs['input_ids'])
    
    # Pass through the first two layers
    for layer in model_part1:
        hidden_states = layer(hidden_states)[0]  # Get the output from each layer

# Serialize and send the intermediate tensor to the server
data = pickle.dumps(hidden_states)

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('172.16.19.89', 10300))  # Adjust host and port as needed
    client_socket.sendall(data)
except Exception as e:
    print(f"Error sending data: {e}")
finally:
    client_socket.close()

print("Intermediate output sent to server.")



C:\Users\Exam\.conda\envs\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.


In [2]:
model_part1

ModuleList(
  (0-1): 2 x BertLayer(
    (attention): BertAttention(
      (self): BertSdpaSelfAttention(
        (query): Linear(in_features=128, out_features=128, bias=True)
        (key): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=128, out_features=128, bias=True)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=128, out_features=512, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=512, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [13]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")

# Example input texts (batch)
texts = [
    "I love this product!",               # Expected label: 1
    "This is the worst experience I've ever had.",  # Expected label: 0
    "The service was fantastic!",         # Expected label: 1
    "Not worth the price.",               # Expected label: 0
    "I would recommend this!"             # Expected label: 1
]

# Example true labels (adjust these based on your actual labels)
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Replace with actual labels

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Get model predictions
with torch.no_grad():
    outputs = tinybert(**inputs)  # Forward pass
    logits = outputs.logits  # Get the logits
    predictions = torch.argmax(logits, dim=-1)  # Get the predicted classes

# Calculate accuracy
accuracy = (predictions == true_labels).float().mean().item()
print(f"Accuracy: {accuracy * 100:.2f}%")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy: 60.00%


In [ ]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")

# Load the IMDb dataset
dataset = load_dataset("imdb")

# Use the test set
test_texts = dataset['test']['text']
test_labels = torch.tensor(dataset['test']['label'])

# Tokenize the input texts
inputs = tokenizer(test_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Get model predictions
with torch.no_grad():
    outputs = tinybert(**inputs)  # Forward pass
    logits = outputs.logits  # Get the logits
    predictions = torch.argmax(logits, dim=-1)  # Get the predicted classes

# Calculate accuracy
accuracy = (predictions == test_labels).float().mean().item()
print(f"Accuracy on IMDb test set: {accuracy * 100:.2f}%")


In [1]:
hi

NameError: name 'hi' is not defined

In [2]:
"ji"


'ji'

In [1]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")

# IMDb test reviews and their expected labels
texts = [
    "I loved this movie! It was fantastic and well-made.",      # Expected label: 1
    "This was the worst film I have ever seen. Very disappointing.",  # Expected label: 0
    "An absolute masterpiece. The performances were outstanding.",  # Expected label: 1
    "I wouldn't recommend this to anyone. It's a waste of time.",  # Expected label: 0
    "Great storyline and excellent direction. A must-watch!",      # Expected label: 1
    "Terrible acting and a boring plot. I regret watching it."     # Expected label: 0
]

# Corresponding true labels for the reviews
true_labels = torch.tensor([1, 0, 1, 0, 1, 0])  # Replace with actual labels

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Get model predictions
with torch.no_grad():
    outputs = tinybert(**inputs)  # Forward pass
    logits = outputs.logits  # Get the logits
    predictions = torch.argmax(logits, dim=-1)  # Get the predicted classes

# Calculate accuracy
accuracy = (predictions == true_labels).float().mean().item()
print(f"Accuracy: {accuracy * 100:.2f}%")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy: 50.00%


In [ ]:
"C:\Users\Exam\Desktop\RAKUTEN MANYA\rakuten-project-main\segmentation\optimized llm segmentation\PSO+Pruning saved model"

In [3]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Example input texts (batch)
texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "The service was fantastic!",
    "Not worth the price.",
    "I would recommend this!"
]

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Forward pass through the first two layers
model_part1 = tinybert.bert.encoder.layer[:2]  # Access the encoder correctly

with torch.no_grad():
    # Get the initial hidden states from the embedding layer
    hidden_states = tinybert.bert.embeddings(inputs['input_ids'])
    
    # Pass through the first two layers
    for layer in model_part1:
        hidden_states = layer(hidden_states)[0]  # Get the output from each layer

# Serialize and send the intermediate tensor to the server
data = pickle.dumps(hidden_states)

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('172.16.19.89', 10300))  # Adjust host and port as needed
    client_socket.sendall(data)
except Exception as e:
    print(f"Error sending data: {e}")
finally:
    client_socket.close()

print("Intermediate output sent to server.")



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.


In [4]:
model_part1

ModuleList(
  (0-1): 2 x BertLayer(
    (attention): BertAttention(
      (self): BertSdpaSelfAttention(
        (query): Linear(in_features=128, out_features=128, bias=True)
        (key): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=128, out_features=128, bias=True)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=128, out_features=512, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=512, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [5]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Input texts
texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "The service was fantastic!",
    "Not worth the price.",
    "I would recommend this!"
]

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Forward pass through embeddings and first encoder layers (Layers 0-5)
with torch.no_grad():
    hidden_states = tinybert.bert.embeddings(inputs['input_ids'])
    for layer in tinybert.bert.encoder.layer[:6]:  # Only Layers 0-5
        hidden_states = layer(hidden_states)[0]

# Serialize and send the intermediate tensor to the server
data = pickle.dumps(hidden_states)

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('172.16.19.89', 10300))  # Replace SERVER_IP with server's IP
    client_socket.sendall(data)
    print("Intermediate output sent to server.")
except Exception as e:
    print(f"Error sending data: {e}")
finally:
    client_socket.close()



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.


In [6]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Sample input texts
texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "The service was fantastic!",
    "Not worth the price.",
    "I would recommend this!"
]

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Forward pass through embeddings and first BertLayer (Layer 0)
with torch.no_grad():
    # Embedding and first BertLayer
    hidden_states = tinybert.bert.embeddings(inputs['input_ids'])
    hidden_states = tinybert.bert.encoder.layer[0](hidden_states)[0]  # Only Layer 0

# Serialize and send the intermediate tensor to the server
data = pickle.dumps(hidden_states)

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('172.16.19.89', 10300))  # Replace SERVER_IP with server's IP address
    client_socket.sendall(data)
    print("Intermediate output sent to server.")
except Exception as e:
    print(f"Error sending data: {e}")
finally:
    client_socket.close()



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.


In [19]:
# Client Side Code (modelpart1)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Example input texts
texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "The service was fantastic!",
    "Not worth the price.",
    "I would recommend this!"
]

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Define modelpart1 (Embeddings and first encoder layer)
modelpart1 = nn.ModuleList([
    tinybert.bert.embeddings,
    tinybert.bert.encoder.layer[0]  # First encoder layer (Layer 0)
])

# Pass through modelpart1
with torch.no_grad():
    # Get embeddings
    hidden_states = modelpart1[0](inputs['input_ids'])
    # Pass through the first encoder layer (Layer 0)
    hidden_states = modelpart1[1](hidden_states)[0]  # Only Layer 0

# Serialize the output and send it to the server
data = pickle.dumps(hidden_states)

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('172.16.19.89', 10300))  # Use the server's IP and port
    client_socket.sendall(data)
    print("Intermediate output sent to server.")
except Exception as e:
    print(f"Error sending data: {e}")
finally:
    client_socket.close()



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.


In [11]:
model_part1

ModuleList(
  (0-1): 2 x BertLayer(
    (attention): BertAttention(
      (self): BertSdpaSelfAttention(
        (query): Linear(in_features=128, out_features=128, bias=True)
        (key): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=128, out_features=128, bias=True)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=128, out_features=512, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=512, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [12]:
tinybert.bert.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 128, padding_idx=0)
  (position_embeddings): Embedding(512, 128)
  (token_type_embeddings): Embedding(2, 128)
  (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [13]:
tinybert.bert.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=512, out_features=128, bias=True)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [14]:
for i in model_part1:
    print(i)

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=512, out_features=128, bias=True)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
   

In [15]:
len(model_part1)

2

In [16]:
model_part1[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=512, out_features=128, bias=True)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [17]:
model_part1[1]

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=512, out_features=128, bias=True)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [18]:
for idx, layer in enumerate(modelpart1):
    print(f"Layer {idx}: {layer}")


Layer 0: BertEmbeddings(
  (word_embeddings): Embedding(30522, 128, padding_idx=0)
  (position_embeddings): Embedding(512, 128)
  (token_type_embeddings): Embedding(2, 128)
  (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
Layer 1: BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()

In [ ]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn
from datasets import load_dataset

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model/")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model/")

# Define modelpart1 (Embeddings and first encoder layer)
modelpart1 = nn.ModuleList([
    tinybert.bert.embeddings,
    tinybert.bert.encoder.layer[0]  # First encoder layer (Layer 0)
])

# Load IMDb dataset without torchtext
dataset = load_dataset("imdb", split="test")
input_texts = [example["text"] for example in dataset]
true_labels = [1 if example["label"] == 1 else 0 for example in dataset]

# Tokenize input text and prepare for processing in batches
inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True, max_length=128)

# Pass through modelpart1
with torch.no_grad():
    hidden_states = modelpart1[0](inputs['input_ids'])
    hidden_states = modelpart1[1](hidden_states)[0]  # First encoder layer

# Serialize intermediate output and true labels
data = pickle.dumps({
    "intermediate_output": hidden_states,
    "true_labels": true_labels
})

# Send data to server
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('172.16.19.89', 10300))  # Replace with your server IP
client_socket.sendall(data)
print("Intermediate output and true labels sent to server.")

# Receive final results from the server
result_data = b""
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    result_data += packet

# Deserialize received results
result = pickle.loads(result_data)
predictions = result["predictions"]
accuracy = result["accuracy"]

# Display results
print(f"Model Predictions: {predictions}")
print(f"Model Accuracy: {accuracy * 100:.2f}%")

client_socket.close()


C:\Users\Exam\.conda\envs\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Intermediate output and true labels sent to server.


In [1]:
import torch

def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.numel() * param.element_size()  # total bytes
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.numel() * buffer.element_size()  # total bytes

    total_size = (param_size + buffer_size) / (1024**2)  # convert bytes to MB
    return total_size

# Example usage
model_size = get_model_size(tinybert)
print(f"TinyBERT model size: {model_size:.2f} MB")


NameError: name 'tinybert' is not defined

In [2]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Initialize the client socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    # Connect to the server
    client_socket.connect(('172.16.19.89', 10300))  # Use the server's IP and port
    print("Connected to the server.")

    while True:
        # Example input texts
        texts = [
            "I love this product!",
            "This is the worst experience I've ever had.",
            "The service was fantastic!",
            "Not worth the price.",
            "I would recommend this!"
        ]

        # Tokenize the input texts
        inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

        # Define modelpart1 (Embeddings and first encoder layer)
        modelpart1 = nn.ModuleList([
            tinybert.bert.embeddings,
            tinybert.bert.encoder.layer[0]  # First encoder layer (Layer 0)
        ])

        # Pass through modelpart1
        with torch.no_grad():
            # Get embeddings
            hidden_states = modelpart1[0](inputs['input_ids'])
            # Pass through the first encoder layer (Layer 0)
            hidden_states = modelpart1[1](hidden_states)[0]  # Only Layer 0

        # Serialize the output and send it to the server
        data = pickle.dumps(hidden_states)
        client_socket.sendall(data)
        print("Intermediate output sent to server.")

        # Wait for the prediction result from the server
        predictions_data = b''
        while True:
            packet = client_socket.recv(4096)
            if not packet:
                break
            predictions_data += packet

        # Deserialize the received predictions
        predictions = pickle.loads(predictions_data)
        print(f"Received predictions from server: {predictions}")

        # Option to stop the process
        user_input = input("Do you want to send more inputs? (yes/no): ").strip().lower()
        if user_input != 'yes':
            client_socket.sendall(pickle.dumps("STOP"))  # Send stop signal
            print("Stopping the client...")
            break

except Exception as e:
    print(f"Error: {e}")
finally:
    client_socket.close()
    print("Connection closed.")



C:\Users\Exam\.conda\envs\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Connected to the server.
Intermediate output sent to server.
Error: [WinError 10054] An existing connection was forcibly closed by the remote host
Connection closed.


In [10]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Get input from the user
texts = []
while True:
    text = input("Enter a sentence (or type 'done' to finish): ")
    if text.lower() == 'done':
        break
    texts.append(text)

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Define modelpart1 (Embeddings and first encoder layer)
modelpart1 = nn.ModuleList([
    tinybert.bert.embeddings,
    tinybert.bert.encoder.layer[0]  # First encoder layer (Layer 0)
])

# Pass through modelpart1
with torch.no_grad():
    # Get embeddings
    hidden_states = modelpart1[0](inputs['input_ids'])
    # Pass through the first encoder layer (Layer 0)
    hidden_states = modelpart1[1](hidden_states)[0]  # Only Layer 0

# Serialize the output and send it to the server
data = pickle.dumps(hidden_states)

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('172.16.19.89', 10300))  # Use the server's IP and port
    client_socket.sendall(data)
    print("Intermediate output sent to server.")
except Exception as e:
    print(f"Error sending data: {e}")
finally:
    client_socket.close()


Enter a sentence (or type 'done' to finish):  chethana is bad
Enter a sentence (or type 'done' to finish):  movie is good
Enter a sentence (or type 'done' to finish):  movie was bad
Enter a sentence (or type 'done' to finish):  done


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.


In [5]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Initialize the client socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    # Connect to the server
    client_socket.connect(('172.16.19.89', 10300))  # Use the server's IP and port
    print("Connected to the server.")

    while True:
        # Get user input
        user_input = input("Enter text to predict (or 'END' to stop): ")
        if user_input.strip().upper() == 'END':
            client_socket.sendall(pickle.dumps("STOP"))  # Send stop signal
            break

        # Tokenize the input text
        inputs = tokenizer(user_input, return_tensors='pt', padding=True, truncation=True)

        # Define modelpart1 (Embeddings and first encoder layer)
        modelpart1 = nn.ModuleList([
            tinybert.bert.embeddings,
            tinybert.bert.encoder.layer[0]  # First encoder layer (Layer 0)
        ])

        # Pass through modelpart1
        with torch.no_grad():
            # Get embeddings
            hidden_states = modelpart1[0](inputs['input_ids'])
            # Pass through the first encoder layer (Layer 0)
            hidden_states = modelpart1[1](hidden_states)[0]  # Only Layer 0

        # Serialize the output and send it to the server
        data = pickle.dumps(hidden_states)
        client_socket.sendall(data)
        print("Intermediate output sent to server.")

        # Wait for the prediction result from the server
        predictions_data = b''
        while True:
            packet = client_socket.recv(4096)
            if not packet:
                break
            predictions_data += packet

        # Deserialize the received predictions
        predictions = pickle.loads(predictions_data)
        print(f"Received predictions from server: {predictions}")

except Exception as e:
    print(f"Error: {e}")
finally:
    client_socket.close()
    print("Connection closed.")



Connected to the server.


Enter text to predict (or 'END' to stop):  movie was good.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.
Error: [WinError 10054] An existing connection was forcibly closed by the remote host
Connection closed.


In [ ]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Define modelpart1 (Embeddings and first encoder layer)
modelpart1 = nn.ModuleList([
    tinybert.bert.embeddings,
    tinybert.bert.encoder.layer[0]  # First encoder layer (Layer 0)
])

while True:
    # Get input from the user
    text = input("Enter a sentence (or type 'done' to finish): ")
    if text.lower() == 'done':
        break
    
    # Tokenize the input text
    inputs = tokenizer([text], return_tensors='pt', padding=True, truncation=True)

    # Pass through modelpart1
    with torch.no_grad():
        # Get embeddings
        hidden_states = modelpart1[0](inputs['input_ids'])
        # Pass through the first encoder layer (Layer 0)
        hidden_states = modelpart1[1](hidden_states)[0]  # Only Layer 0

    # Serialize the output
    data = pickle.dumps(hidden_states)

    try:
        # Establish a connection to the server for each input
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('172.16.19.89', 10300))  # Use the server's IP and port
        client_socket.sendall(data)
        print("Intermediate output sent to server.")

        # Receive the response from the server
        result_data = b""
        while True:
            packet = client_socket.recv(4096)
            if not packet:
                break
            result_data += packet

        # Deserialize and display the result from the server
        result = pickle.loads(result_data)
        print(f"Server Response - Predictions: {result['predictions']}")

    except Exception as e:
        print(f"Error communicating with the server: {e}")
    finally:
        client_socket.close()


Enter a sentence (or type 'done' to finish):  manya is good


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.
Error communicating with the server: [WinError 10054] An existing connection was forcibly closed by the remote host


In [ ]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Define modelpart1 (Embeddings and first encoder layer)
modelpart1 = nn.ModuleList([
    tinybert.bert.embeddings,
    tinybert.bert.encoder.layer[0]  # First encoder layer (Layer 0)
])

# Set up the socket connection to the server
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('172.16.19.89', 10300))  # Replace with your server IP and port

try:
    while True:
        # Get input from the user
        text = input("Enter a sentence (or type 'done' to finish): ")
        if text.lower() == 'done':
            break  # Exit loop if user finishes input
        
        # Tokenize the input text
        inputs = tokenizer([text], return_tensors='pt', padding=True, truncation=True)

        # Pass through modelpart1
        with torch.no_grad():
            hidden_states = modelpart1[0](inputs['input_ids'])  # Embeddings
            hidden_states = modelpart1[1](hidden_states)[0]     # First encoder layer (Layer 0)

        # Serialize the output and send it to the server
        data = pickle.dumps(hidden_states)
        client_socket.sendall(data)
        print("Intermediate output sent to server.")
        
        # Wait for server response (predictions or any feedback)
        result_data = b""
        while True:
            packet = client_socket.recv(4096)
            if not packet:
                break
            result_data += packet

        # Deserialize received result
        result = pickle.loads(result_data)
        print(f"Server Response: {result}")

except Exception as e:
    print(f"Error during communication: {e}")
finally:
    # Close the connection when done
    client_socket.close()
    print("Connection closed.")


C:\Users\Exam\.conda\envs\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Enter a sentence (or type 'done' to finish):  manya is good


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.


In [6]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning saved model")
tokenizer = BertTokenizer.from_pretrained("./PSO+Pruning saved model")

# Get input from the user
texts = []
while True:
    text = input("Enter a sentence (or type 'done' to finish): ")
    if text.lower() == 'done':
        break
    texts.append(text)

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Define modelpart1 (Embeddings and first encoder layer)
modelpart1 = nn.ModuleList([
    tinybert.bert.embeddings,
    tinybert.bert.encoder.layer[0]  # First encoder layer (Layer 0)
])

# Pass through modelpart1
with torch.no_grad():
    # Get embeddings
    hidden_states = modelpart1[0](inputs['input_ids'])
    # Pass through the first encoder layer (Layer 0)
    hidden_states = modelpart1[1](hidden_states)[0]  # Only Layer 0

# Serialize the output and send it to the server
data = pickle.dumps(hidden_states)

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('172.16.19.89', 10300))  # Use the server's IP and port
    client_socket.sendall(data)
    print("Intermediate output sent to server.")
except Exception as e:
    print(f"Error sending data: {e}")
finally:
    client_socket.close()


Enter a sentence (or type 'done' to finish):  Movie is good
Enter a sentence (or type 'done' to finish):  Movie was amazing
Enter a sentence (or type 'done' to finish):  Movie was terrible
Enter a sentence (or type 'done' to finish):  done


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.
